# Cable Bridge 3D

In [124]:
''' Inputs '''
tower_base = 6                   # Metres below the origin
tower_height_to_first_stay = 10  # Metres above the origin
cable_spacing_tower = 1          # Metre increments

cable_spacing_forwards = [5,7.5,7.5,7.5,7.5,7.5,7.5,7.5]
cable_spacing_backwards = [5,5,5,5,5,5,5,5]

deck_width = 5.0
deck_depth = 0.6

tower_width_base = 2.0
tower_depth_base = 2.6
tower_width_top = 2.0
tower_depth_top = 2.6
tower_spacing = 7
stay_dia = 0.15

In [125]:
''' Data Checks'''
assert len(cable_spacing_backwards) == len(cable_spacing_forwards)

In [126]:
import numpy as np
import math
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0") # Connect to LUSAS Modeller

# If there is not currently a model open, create one
if not lusas.existsDatabase():
    lusas.newProject("Structural", "Cable Bridge 3D")
else:
    import sys; sys.path.append('../') # Locate the helpers file
    from m100_Tools_And_Helpers import Helpers
    Helpers.initialise(lusas)
    Helpers.reset_database_contents(lusas.database())

db = lusas.database() # Reference to the model database for convenience

# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")

# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [127]:
''' Create Model Points '''

#point_origin = Helpers.create_point(0,0,0)
tower1_points = [Helpers.create_point(0, tower_spacing/2, -tower_base)]
tower2_points = [Helpers.create_point(0,-tower_spacing/2, -tower_base)]
z = tower_height_to_first_stay
for i in range(0, len(cable_spacing_forwards)):
    tower1_points.append(Helpers.create_point(0, tower_spacing/2,z))
    tower2_points.append(Helpers.create_point(0,-tower_spacing/2,z))
    z+=cable_spacing_tower
tower_points = []
tower_points.append(tower1_points)
tower_points.append(tower2_points)

deck_points_forwards = []
deck_points_forwards.append([Helpers.create_point(x, tower_spacing/2,0) for x in np.add.accumulate(cable_spacing_forwards)])
deck_points_forwards.append([Helpers.create_point(x,-tower_spacing/2,0) for x in np.add.accumulate(cable_spacing_forwards)])

deck_points_backwards = []
deck_points_backwards.append([Helpers.create_point(-x, tower_spacing/2,0) for x in np.add.accumulate(cable_spacing_backwards)])
deck_points_backwards.append([Helpers.create_point(-x,-tower_spacing/2,0) for x in np.add.accumulate(cable_spacing_backwards)])

deck_points = []
deck_points.append(deck_points_backwards[0][::-1] + deck_points_forwards[0])
deck_points.append(deck_points_backwards[1][::-1] + deck_points_forwards[1])

In [128]:
print(len(tower_points))
print(len(tower_points[0]))
print(len(tower_points[1]))

2
9
9


In [129]:
''' Create model lines '''

tower_lines = []
for iy in [0,1]:
    tower_lines.append(list())
    for i in range(0, len(tower_points[iy])-1):
        tower_lines[iy].append(Helpers.create_line_from_points(tower_points[iy][i], tower_points[iy][i+1]))

deck_lines = []
for iy in [0,1]:
    deck_lines.append(list())
    for i in range(0, len(deck_points[iy])-1):
        deck_lines[iy].append(Helpers.create_line_from_points(deck_points[iy][i], deck_points[iy][i+1])) 

back_stays = []
for iy in [0,1]:
    back_stays.append(list())
    for i in range(0, len(deck_points_backwards[iy])-1):
        back_stays[iy].append(Helpers.create_line_from_points(deck_points_backwards[iy][i], tower_points[iy][i+2])) 

front_stays = []
for iy in [0,1]:
    front_stays.append(list())
    for i in range(0, len(deck_points_forwards[iy])-1):
        front_stays[iy].append(Helpers.create_line_from_points(deck_points_forwards[iy][i], tower_points[iy][i+2])) 

cross_lines = []
for i in range(0, len(deck_points[0])):
    cross_lines.append(Helpers.create_line_from_points(deck_points[0][i], deck_points[1][i])) 

In [130]:
''' Create Geometric Sections '''
deck_section_attr = Helpers.create_rectangular_section(db, "Deck", deck_width, deck_depth)
tower_section_attr = Helpers.create_rectangular_section(db, "Tower", tower_width_base, tower_depth_base)
stay_section_attr = Helpers.create_circular_section(db, "Stay", stay_dia)

deck_section_attr.assignTo(deck_lines[0])
deck_section_attr.assignTo(deck_lines[1])
# TODO calculate based on spacing
deck_section_attr.assignTo(cross_lines)

tower_section_attr.assignTo(tower_lines[0])
tower_section_attr.assignTo(tower_lines[1])
stay_section_attr.assignTo(back_stays[0])
stay_section_attr.assignTo(back_stays[1])
stay_section_attr.assignTo(front_stays[0])
stay_section_attr.assignTo(front_stays[1])

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x1794612176288>

In [131]:
''' Create Material Attributes'''

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000_000, 0.3, 7.8, 10e-6)

# Concrete Material attribute
concrete_material = db.createIsotropicMaterial("Concrete", 30_000_000, 0.2, 2.4, 10e-6)

steel_material.assignTo(back_stays[0])
steel_material.assignTo(back_stays[1])
steel_material.assignTo(front_stays[0])
steel_material.assignTo(front_stays[1])
concrete_material.assignTo(deck_lines[0])
concrete_material.assignTo(deck_lines[1])
concrete_material.assignTo(cross_lines)

concrete_material.assignTo(tower_lines[0])
concrete_material.assignTo(tower_lines[1])

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x1794602845296>

In [132]:
''' Support attributes ''' 
fixed = db.createSupportStructural("Fixed in XY").setStructural("R", "R", "F", "R", "R", "R", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in Z").setStructural("F", "F", "R", "F", "F", "F", "F", "F", "C", "F")

fixed.assignTo(tower_points[0][0])
fixed.assignTo(tower_points[1][0])
slide.assignTo(deck_points_backwards[0][-1])
slide.assignTo(deck_points_backwards[1][-1])
slide.assignTo(deck_points_forwards[0][-1])
slide.assignTo(deck_points_forwards[1][-1])

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x1794604115312>

In [133]:
''' Mesh Attributes'''
# Cables modelles with single bar element
cable_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BRS2", 1)
# Deck and tower modelled with beam elements
beam_mesh_attr = db.createMeshLine("Beam mesh 1m").setSize("BMI21", 1)

cable_mesh_attr.assignTo(back_stays[0])
cable_mesh_attr.assignTo(back_stays[1])
cable_mesh_attr.assignTo(front_stays[0])
cable_mesh_attr.assignTo(front_stays[1])
beam_mesh_attr.assignTo(deck_lines[0])
beam_mesh_attr.assignTo(deck_lines[1])
beam_mesh_attr.assignTo(tower_lines[0])
beam_mesh_attr.assignTo(tower_lines[1])
beam_mesh_attr.assignTo(cross_lines)

# Create the model mesh
db.updateMesh()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFDatabase instance at 0x1794604116464>

# Loads

In [134]:
# Automatic gravity in loadcase 1
loadset = db.getLoadset("Loadcase 1").setName("Self Weight")
win32.CastTo(loadset, "IFLoadcase").addGravity(True)